In [1]:
import pandas as pd
import numpy as np
import re

In [4]:
# Indicate file path and load it as df 
file_path = '../Resources/scraped_cat_food_20240122.csv'
cat_kibble_df = pd.read_csv(file_path)

cat_kibble_df.head()

,Product Name,Price,Bag Size,Description,Ingredients,Guaranteed Analysis,Scraping Date,Product Link
0,performatrin Ultra Grain-Free Original Recipe ...,$69.99,12 lb,performatrin Ultra Grain-free Original Recipe ...,"Turkey, Chicken Meal, Potato, Duck Meal, Salmo...",Crude Protein (min.) 40.0% Crude Fat (min.) 18...,2024-01-22,https://www.petvalu.ca/product/performatrin-ul...
1,performatrin Ultra Wholesome Grains Healthy We...,$59.99,12 lb,performatrin Ultra Wholesome Grains Healthy We...,"Salmon, Salmon Meal, Chicken Meal, Brown Rice,...",Crude Protein (min.) 33.0% Crude Fat (min.) 10...,2024-01-22,https://www.petvalu.ca/product/performatrin-ul...
2,performatrin Ultra Grain-Free Senior Recipe Ca...,$69.99,12 lb,performatrin Ultra Grain-Free Senior Recipe Ca...,"Turkey, Peas, Lentils, Turkey Meal, Chickpeas,...",Crude Protein (min.) 30.0% Crude Fat (min.) 15...,2024-01-22,https://www.petvalu.ca/product/performatrin-ul...
3,performatrin Ultra Grain-Free Ocean Recipe Cat...,$65.99,12 lb,performatrin Ultra Grain-free Ocean Recipe Cat...,"Ocean Whitefish, Salmon Meal, Herring Meal, Pe...",Crude Protein (min.) 38.0% Crude Fat (min.) 18...,2024-01-22,https://www.petvalu.ca/product/performatrin-ul...
4,Royal Canin Indoor Adult Cat Food,$113.99,15 lb,"Indoor cats spend more time sleeping, eating a...","Chicken Meal, Corn, Brewers Rice, Corn Gluten ...",Crude Protein (min) 27.0% Crude Fat (min) 11.0...,2024-01-22,https://www.petvalu.ca/product/royal-canin-ind...


In [5]:
# Get the number of rows & column names
num_rows = cat_kibble_df.shape[0]
column_names = cat_kibble_df.columns.tolist()

print(f"Number of rows: {num_rows}")
print(f"Column names: {column_names}")

Number of rows: 153
Column names: ['Product Name', 'Price', 'Bag Size', 'Description', 'Ingredients', 'Guaranteed Analysis', 'Scraping Date', 'Product Link']


In [6]:
# Drop empty rows
cat_kibble_df.dropna(inplace=True)
num_rows_cleaned = cat_kibble_df.shape[0]

print(f"Number of rows after removing empty rows: {num_rows_cleaned}")

Number of rows after removing empty rows: 153


In [7]:
# Separate the Brand Name from the Kibble Name
brand_names = ["ACANA", "Blue Buffalo", "Fussie Cat", "Go! Solutions", "Hill's Science Diet", "Iams", 
               "Lifetime", "Lovibles", "Meow Mix", "Merrick", "Natural Balance", "Now Fresh", "Open Farm", 
               "ORIJEN", "performatrin NATURALS", "performatrin Prime", "performatrin Ultra", 
               "Purina", "Purina Pro Plan", "Royal Canin", "Stella & Chewy's", 
               "Summit", "Taste of the Wild", "The Honest Kitchen", "Tiki Cat", "Whiskas"]

# Create a pattern to match brand names
brand_pattern = '|'.join(brand_names)

# Extract brand and kibble name using regex
cat_kibble_df[['Brand', 'Kibble Name']] = cat_kibble_df['Product Name'].str.extract(f'({brand_pattern})\s?(.*)', expand=True)

### SEPARATES THE BRANDS EXCEPT FOR WHEN THE BRAND NAME REPEATS LIKE "performatrin", "performatrin NATURALS", "Purina", "Purina Pro Plan"

In [8]:
# Separate sub brands
sub_brand_names = ["NATURALS", "Prime", "Ultra", "Pro Plan"]

# Function to check if Kibble Name contains any sub-brand name
def extract_sub_brand(kibble_name):
    return next((sub_brand for sub_brand in sub_brand_names if isinstance(kibble_name, str) and sub_brand in kibble_name), None)

# Create 'Sub Brand' column
cat_kibble_df['Sub Brand'] = cat_kibble_df['Kibble Name'].apply(extract_sub_brand)

# Update 'Brand' column by joining with 'Sub Brand' if applicable
cat_kibble_df['Brand'] = np.where(~cat_kibble_df['Sub Brand'].isna(), cat_kibble_df['Brand'] + ' ' + cat_kibble_df['Sub Brand'], cat_kibble_df['Brand'])

# Remove sub-brand from 'Kibble Name'
def remove_sub_brand(kibble_name, sub_brand):
    if isinstance(kibble_name, str) and isinstance(sub_brand, str):
        return kibble_name.replace(sub_brand, '').strip()
    return kibble_name

cat_kibble_df['Kibble Name'] = cat_kibble_df.apply(lambda row: remove_sub_brand(row['Kibble Name'], row['Sub Brand']), axis=1)

In [9]:
# Remove '$' from 'Price' values and convert to numeric
cat_kibble_df['Price($)'] = pd.to_numeric(cat_kibble_df['Price'].str.replace(r'[^0-9.]', '', regex=True), errors='coerce')

# Some Values in "Price" look like this $49.99$56.99 due to being on sale while scraping  
# Identify rows with the specified format in 'Price'
odd_price_indices = cat_kibble_df[cat_kibble_df['Price'].astype(str).str.contains(r'\$\d+\.\d+\$\d+\.\d+')].index

# Replace the first number in 'Price' column for the identified rows with an empty string
cat_kibble_df.loc[odd_price_indices, 'Price($)'] = cat_kibble_df.loc[odd_price_indices, 'Price'].astype(str).replace(to_replace=r'^(\$\d+\.\d+)\$(\d+\.\d+)$', value=r'\2', regex=True).astype(float)

#dog_kibble_df.loc[odd_price_indices, ['Price($)']]

In [10]:
# Separate Top 5 ingridients 
cat_kibble_df['TOP 5 Ingredients'] = cat_kibble_df['Ingredients'].apply(lambda x: ', '.join(str(x).split(', ')[:5]))

# Extract only the numerical part from 'Bag Size'
cat_kibble_df['Bag Size(lbs)'] = cat_kibble_df['Bag Size'].str.extract(r'(\d+)', expand=False)

# Convert 'Bag Size(lbs)' to numeric values
cat_kibble_df['Bag Size(lbs)'] = pd.to_numeric(cat_kibble_df['Bag Size(lbs)'], errors='coerce')

# Create 'Bag Size(kg)' column by converting 'Bag Size(lbs)' to kilograms (1 lb = 0.453592 kg)
cat_kibble_df['Bag Size(kg)'] = round(cat_kibble_df['Bag Size(lbs)'] * 0.453592, 2)

cat_kibble_df.head()

,Product Name,Price,Bag Size,Description,Ingredients,Guaranteed Analysis,Scraping Date,Product Link,Brand,Kibble Name,Sub Brand,Price($),TOP 5 Ingredients,Bag Size(lbs),Bag Size(kg)
0,performatrin Ultra Grain-Free Original Recipe ...,$69.99,12 lb,performatrin Ultra Grain-free Original Recipe ...,"Turkey, Chicken Meal, Potato, Duck Meal, Salmo...",Crude Protein (min.) 40.0% Crude Fat (min.) 18...,2024-01-22,https://www.petvalu.ca/product/performatrin-ul...,performatrin Ultra,Grain-Free Original Recipe Cat Food,None,69.99,"Turkey, Chicken Meal, Potato, Duck Meal, Salmo...",12,5.44
1,performatrin Ultra Wholesome Grains Healthy We...,$59.99,12 lb,performatrin Ultra Wholesome Grains Healthy We...,"Salmon, Salmon Meal, Chicken Meal, Brown Rice,...",Crude Protein (min.) 33.0% Crude Fat (min.) 10...,2024-01-22,https://www.petvalu.ca/product/performatrin-ul...,performatrin Ultra,Wholesome Grains Healthy Weight Recipe with Sa...,None,59.99,"Salmon, Salmon Meal, Chicken Meal, Brown Rice,...",12,5.44
2,performatrin Ultra Grain-Free Senior Recipe Ca...,$69.99,12 lb,performatrin Ultra Grain-Free Senior Recipe Ca...,"Turkey, Peas, Lentils, Turkey Meal, Chickpeas,...",Crude Protein (min.) 30.0% Crude Fat (min.) 15...,2024-01-22,https://www.petvalu.ca/product/performatrin-ul...,performatrin Ultra,Grain-Free Senior Recipe Cat Food,None,69.99,"Turkey, Peas, Lentils, Turkey Meal, Chickpeas",12,5.44
3,performatrin Ultra Grain-Free Ocean Recipe Cat...,$65.99,12 lb,performatrin Ultra Grain-free Ocean Recipe Cat...,"Ocean Whitefish, Salmon Meal, Herring Meal, Pe...",Crude Protein (min.) 38.0% Crude Fat (min.) 18...,2024-01-22,https://www.petvalu.ca/product/performatrin-ul...,performatrin Ultra,Grain-Free Ocean Recipe Cat Food,None,65.99,"Ocean Whitefish, Salmon Meal, Herring Meal, Pe...",12,5.44
4,Royal Canin Indoor Adult Cat Food,$113.99,15 lb,"Indoor cats spend more time sleeping, eating a...","Chicken Meal, Corn, Brewers Rice, Corn Gluten ...",Crude Protein (min) 27.0% Crude Fat (min) 11.0...,2024-01-22,https://www.petvalu.ca/product/royal-canin-ind...,Royal Canin,Indoor Adult Cat Food,None,113.99,"Chicken Meal, Corn, Brewers Rice, Corn Gluten ...",15,6.80


In [11]:
# Price per kg and price per lbs 
cat_kibble_df['Price per 1kg'] = round(cat_kibble_df['Price($)'] / cat_kibble_df['Bag Size(kg)'],2)

cat_kibble_df['Price per 1lbs'] = round(cat_kibble_df['Price($)'] / cat_kibble_df['Bag Size(lbs)'],2)

In [12]:
# Rename and Drop columns
cat_kibble_df.drop(['Product Name', 'Sub Brand', 'Price', 'Bag Size'], axis=1, inplace=True)

cat_kibble_df.rename(columns={'Ingredients': 'Full Ingredient List'}, inplace=True)


In [13]:
# Reorder columns
new_order = ['Brand', 'Kibble Name', 'Bag Size(lbs)', 'Bag Size(kg)', 'Price($)', 'Price per 1lbs', 'Price per 1kg', 'TOP 5 Ingredients', 'Description',
             'Full Ingredient List', 'Guaranteed Analysis', 'Product Link', 'Scraping Date']

cat_kibble_df = cat_kibble_df[new_order]

In [14]:
cat_kibble_df.head()

,Brand,Kibble Name,Bag Size(lbs),Bag Size(kg),Price($),Price per 1lbs,Price per 1kg,TOP 5 Ingredients,Description,Full Ingredient List,Guaranteed Analysis,Product Link,Scraping Date
0,performatrin Ultra,Grain-Free Original Recipe Cat Food,12,5.44,69.99,5.83,12.87,"Turkey, Chicken Meal, Potato, Duck Meal, Salmo...",performatrin Ultra Grain-free Original Recipe ...,"Turkey, Chicken Meal, Potato, Duck Meal, Salmo...",Crude Protein (min.) 40.0% Crude Fat (min.) 18...,https://www.petvalu.ca/product/performatrin-ul...,2024-01-22
1,performatrin Ultra,Wholesome Grains Healthy Weight Recipe with Sa...,12,5.44,59.99,5.00,11.03,"Salmon, Salmon Meal, Chicken Meal, Brown Rice,...",performatrin Ultra Wholesome Grains Healthy We...,"Salmon, Salmon Meal, Chicken Meal, Brown Rice,...",Crude Protein (min.) 33.0% Crude Fat (min.) 10...,https://www.petvalu.ca/product/performatrin-ul...,2024-01-22
2,performatrin Ultra,Grain-Free Senior Recipe Cat Food,12,5.44,69.99,5.83,12.87,"Turkey, Peas, Lentils, Turkey Meal, Chickpeas",performatrin Ultra Grain-Free Senior Recipe Ca...,"Turkey, Peas, Lentils, Turkey Meal, Chickpeas,...",Crude Protein (min.) 30.0% Crude Fat (min.) 15...,https://www.petvalu.ca/product/performatrin-ul...,2024-01-22
3,performatrin Ultra,Grain-Free Ocean Recipe Cat Food,12,5.44,65.99,5.50,12.13,"Ocean Whitefish, Salmon Meal, Herring Meal, Pe...",performatrin Ultra Grain-free Ocean Recipe Cat...,"Ocean Whitefish, Salmon Meal, Herring Meal, Pe...",Crude Protein (min.) 38.0% Crude Fat (min.) 18...,https://www.petvalu.ca/product/performatrin-ul...,2024-01-22
4,Royal Canin,Indoor Adult Cat Food,15,6.80,113.99,7.60,16.76,"Chicken Meal, Corn, Brewers Rice, Corn Gluten ...","Indoor cats spend more time sleeping, eating a...","Chicken Meal, Corn, Brewers Rice, Corn Gluten ...",Crude Protein (min) 27.0% Crude Fat (min) 11.0...,https://www.petvalu.ca/product/royal-canin-ind...,2024-01-22


In [15]:
# Remove duplicate rows
cat_kibble_df.drop_duplicates(inplace=True)

# Check for missing values in other columns
missing_values = cat_kibble_df.isnull().sum()
missing_values

Brand                   0
Kibble Name             0
Bag Size(lbs)           0
Bag Size(kg)            0
Price($)                0
Price per 1lbs          0
Price per 1kg           0
TOP 5 Ingredients       0
Description             0
Full Ingredient List    0
Guaranteed Analysis     0
Product Link            0
Scraping Date           0
dtype: int64

In [16]:
cat_kibble_df.dtypes

Brand                    object
Kibble Name              object
Bag Size(lbs)             int64
Bag Size(kg)            float64
Price($)                float64
Price per 1lbs          float64
Price per 1kg           float64
TOP 5 Ingredients        object
Description              object
Full Ingredient List     object
Guaranteed Analysis      object
Product Link             object
Scraping Date            object
dtype: object

In [17]:
cat_kibble_df['Scraping Date'] = pd.to_datetime(cat_kibble_df['Scraping Date'])
cat_kibble_df.dtypes

Brand                           object
Kibble Name                     object
Bag Size(lbs)                    int64
Bag Size(kg)                   float64
Price($)                       float64
Price per 1lbs                 float64
Price per 1kg                  float64
TOP 5 Ingredients               object
Description                     object
Full Ingredient List            object
Guaranteed Analysis             object
Product Link                    object
Scraping Date           datetime64[ns]
dtype: object

In [18]:
# Specify the file path to save the cleaned DataFrame
output_file_path = '../Resources/cleaned_cat_food_data.csv'

cat_kibble_df.to_csv(output_file_path, index=False)

print(f"Data saved to: {output_file_path}")

Data saved to: ../Resources/cleaned_cat_food_data.csv
